# Week 5 and 6 - Assignment 6
#### Team 5

**Goal:** Classify new "test" documents using already classified "training" documents.

For this project, the dataset used was from the CrowdFlower library (https://www.crowdflower.com/data-for-everyone/), using the "Twitter sentiment analysis: Self-driving cars" dataset. This dataset includes posts from Twitter regarding users' opinions of self-driving cars, the tweets' classification as relevant to the topic or not.

# Importing data

#### Seeing what we're working with

In [1]:
import nltk 
import pandas as pd
import sys
from nltk.classify.scikitlearn import SklearnClassifier
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize.api import TokenizerI
from nltk.tokenize.toktok import ToktokTokenizer
toktok = ToktokTokenizer()

In [2]:
data = pd.io.parsers.read_csv("https://raw.githubusercontent.com/Galanopoulog/DATA620-Assignment-6/master/Self-Driving-Cars.csv")
data = data.drop(data.columns[[1,2,3,4,7,8,9]], axis=1)
data.text.replace({r'[^\x00-\x7F]+':''}, regex=True, inplace=True) #remove ascii characters
data.head(11)

,_unit_id,sentiment,sentiment:confidence,text
0,724227031,5,0.7579,Two places I'd invest all my money if I could:...
1,724227032,5,0.8775,Awesome! Google driverless cars will help the ...
2,724227033,2,0.6805,If Google maps can't keep up with road constru...
3,724227034,2,0.8820,Autonomous cars seem way overhyped given the t...
4,724227035,3,1.0000,Just saw Google self-driving car on I-34. It w...
5,724227036,3,1.0000,Will driverless cars eventually replace taxi d...
6,724227037,not_relevant,0.5367,Chicago metro expected to be fully autonomous ...
7,724227038,not_relevant,0.6548,I love the infotainment system in my new car. ...
8,724227039,5,0.7187,Autonomous vehicles could reduce traffic fatal...
9,724227040,1,0.6412,Driverless cars are not worth the risk. Don't...


In [3]:
len(data)

7156

In [4]:
print(data.dtypes)

_unit_id                  int64
sentiment                object
sentiment:confidence    float64
text                     object
dtype: object


In [5]:
# Count relevant and irrelevant
rel_count = len(data[data.sentiment!="not_relevant"])
irrel_count = len(data[data.sentiment=="not_relevant"])
neut_count = len(data[data.sentiment=="3"])

print "Relevant: %d" %rel_count
print "Not_Relevant: %d" %irrel_count
print "Neutral: %d" %neut_count

Relevant: 6943
Not_Relevant: 213
Neutral: 4245


Two key things to note is that 1) the tweets and sentiment values are objects, which may need to be changed in the future for manipulations and 2) the number of non-relevant tweets is significantly smaller than the relevant portion, so splitting the data into training and test sets should probably be more methodical than just randomly shuffling the data. Also, it will be interesting to see the difference between the relevant neutral tweets and the irrelevant ones.

#### Splitting the data

The data is split into the training set, the evaluation set and the final set that will be used for testing.

In [6]:
# What sizes should our sets be?
train_set = int(0.7 * len(data))
dev_set = int(.15 * len(data))
final_set = len(data) - train_set - dev_set
Total = train_set + dev_set + final_set

print "Training set: %d" %train_set
print "Evaluation set: %d" %dev_set
print "Testing set: %d" %final_set
print "Total: %d" %Total

Training set: 5009
Evaluation set: 1073
Testing set: 1074
Total: 7156


In [7]:
# Let's split our data into relevant and non-relevant parts.
relevant = data[data.sentiment!="not_relevant"]
nonrelevant = data[data.sentiment=="not_relevant"]

# Let's take 70%, 15% and the 15% of each
rel_train = int(0.7 * len(relevant))
rel_dev = int(.15 * len(relevant))
rel_fin = len(relevant) - rel_train - rel_dev
rel_Total = rel_train + rel_dev + rel_fin

irrel_train = int(0.7 * len(nonrelevant))
irrel_dev = int(.15 * len(nonrelevant))
irrel_fin = len(nonrelevant) - irrel_train - irrel_dev
irrel_Total = irrel_train + irrel_dev + irrel_fin

print "Training set: %d" %rel_train
print "Evaluation set: %d" %rel_dev
print "Testing set: %d" %rel_fin
print "Total: %d" %rel_Total
print "----------------------"
print "Training set: %d" %irrel_train
print "Evaluation set: %d" %irrel_dev
print "Testing set: %d" %irrel_fin
print "Total: %d" %irrel_Total

Training set: 4860
Evaluation set: 1041
Testing set: 1042
Total: 6943
----------------------
Training set: 149
Evaluation set: 31
Testing set: 33
Total: 213


In [8]:
# Make sets for relevant data
rel_train = relevant[0:4860]
rel_dev = relevant[4860:5901]
rel_fin = relevant[5901:6943]

# Make sets for non-relevant data
irrel_train = nonrelevant[0:149]
irrel_dev = nonrelevant[149:180]
irrel_fin = nonrelevant[180:231]

# Combine sets by set types
train_set = pd.concat([rel_train, irrel_train], axis=0)
dev_set = pd.concat([rel_dev, irrel_dev], axis=0)
final_set = pd.concat([rel_fin, irrel_fin], axis=0)

# Determine lengths and compare to set size estimates
print "Training set: %d" %len(train_set)
print "Evaluation set: %d" %len(dev_set)
print "Testing set: %d" %len(final_set)

Training set: 5009
Evaluation set: 1072
Testing set: 1075


All sets now have an evenly distributed number of relevant and non-relevant entries and match the 70%, 15%, 15% data distribution that was considered optimal originally.

# Analyze and Categorize Data

In [9]:
from textblob.classifiers import NaiveBayesClassifier

In [10]:
train_set[0:10]

,_unit_id,sentiment,sentiment:confidence,text
0,724227031,5,0.7579,Two places I'd invest all my money if I could:...
1,724227032,5,0.8775,Awesome! Google driverless cars will help the ...
2,724227033,2,0.6805,If Google maps can't keep up with road constru...
3,724227034,2,0.8820,Autonomous cars seem way overhyped given the t...
4,724227035,3,1.0000,Just saw Google self-driving car on I-34. It w...
5,724227036,3,1.0000,Will driverless cars eventually replace taxi d...
8,724227039,5,0.7187,Autonomous vehicles could reduce traffic fatal...
9,724227040,1,0.6412,Driverless cars are not worth the risk. Don't...
10,724227041,3,0.9184,"Driverless cars are now legal in Florida, Cali..."
11,724227610,3,1.0000,Audi is the first carmaker to get a license fr...


In [11]:
# Making tuples for classification
subset = train_set[['text', 'sentiment']]
tuples = [tuple(x) for x in subset.values]

train_set[['text']][0:10]
subset[['text']]
len(tuples)

5009

In [12]:
# Tokenizing words
all_words = toktok.tokenize(subset[['text']] )

t = [({word: (word in (x[0])) for word in all_words}, x[1]) for x in tuples]
len(t)

5009

In [45]:
# Classifying Methods

# Naive Bayes
classifier = nltk.NaiveBayesClassifier.train(t)
classifier.show_most_informative_features(20)

Most Informative Features
                    Damn = True                1 : 3      =     41.9 : 1.0
                    Fuck = True                1 : 3      =     41.9 : 1.0
                    cool = True                5 : 3      =     33.4 : 1.0
                 Awesome = True                5 : 3      =     32.7 : 1.0
                    seem = True                1 : 3      =     23.3 : 1.0
                  reduce = True                5 : 3      =     22.4 : 1.0
                       c = False          not_re : 2      =     20.9 : 1.0
                tracking = True           not_re : 3      =     20.4 : 1.0
         _______________ = True           not_re : 3      =     20.4 : 1.0
                     Dor = True           not_re : 3      =     20.4 : 1.0
               surprised = True           not_re : 3      =     20.4 : 1.0
                 On-Road = True           not_re : 3      =     20.4 : 1.0
                  Second = True           not_re : 3      =     20.4 : 1.0

In [17]:
# Other Methods to Consider

# MNB
MNB_classifier = SklearnClassifier(MultinomialNB())
MNB_classifier.train(t)

# BernoulliNB
BernoulliNB_classifier = SklearnClassifier(BernoulliNB())
BernoulliNB_classifier.train(t)

# Logistic Regression
LogisticRegression_classifier = SklearnClassifier(LogisticRegression())
LogisticRegression_classifier.train(t)

# SGDC
SGDClassifier_classifier = SklearnClassifier(SGDClassifier())
SGDClassifier_classifier.train(t)

# Linear SVC
LinearSVC_classifier = SklearnClassifier(LinearSVC())
LinearSVC_classifier.train(t)

print "Percent Accuracy:"
print "------------------"
print "Naive Bayes: %d" %round((nltk.classify.accuracy(classifier, t))*100, 2)
print "MNB: %d" %round((nltk.classify.accuracy(MNB_classifier, t))*100, 2)
print "BernoulliNB: %d" %round((nltk.classify.accuracy(BernoulliNB_classifier, t))*100, 2)
print "LogisticRegression: %d" %round((nltk.classify.accuracy(LogisticRegression_classifier, t))*100, 2)
print "SGDClassifier: %d" %round((nltk.classify.accuracy(SGDClassifier_classifier, t))*100, 2)
print "LinearSVC: %d" %round((nltk.classify.accuracy(LinearSVC_classifier, t))*100, 2)

Percent Accuracy:
------------------
Naive Bayes: 52
MNB: 60
BernoulliNB: 53
LogisticRegression: 65
SGDClassifier: 49
LinearSVC: 65


Out of all these models, the one that has the highest accuracy is the Linear SVC model. As interesting as it is to see the differences in each of the values within relevant tweets, it may be best to change the values of relevant (from 5 to 1) to one single value to better see the difference between relevant and non-relevant tweets.

#### Changing non-relevant values to "relevant"

In [18]:
# Changing values
train_set.sentiment[train_set.sentiment != "not_relevant"] = "relevant"
final_set.sentiment[final_set.sentiment != "not_relevant"] = "relevant" # testing dataset
train_set[:10]

C:\Users\Georgia\Anaconda2\envs\gl-env\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\Users\Georgia\Anaconda2\envs\gl-env\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


,_unit_id,sentiment,sentiment:confidence,text
0,724227031,relevant,0.7579,Two places I'd invest all my money if I could:...
1,724227032,relevant,0.8775,Awesome! Google driverless cars will help the ...
2,724227033,relevant,0.6805,If Google maps can't keep up with road constru...
3,724227034,relevant,0.8820,Autonomous cars seem way overhyped given the t...
4,724227035,relevant,1.0000,Just saw Google self-driving car on I-34. It w...
5,724227036,relevant,1.0000,Will driverless cars eventually replace taxi d...
8,724227039,relevant,0.7187,Autonomous vehicles could reduce traffic fatal...
9,724227040,relevant,0.6412,Driverless cars are not worth the risk. Don't...
10,724227041,relevant,0.9184,"Driverless cars are now legal in Florida, Cali..."
11,724227610,relevant,1.0000,Audi is the first carmaker to get a license fr...


In [19]:
train_set.tail()

,_unit_id,sentiment,sentiment:confidence,text
4638,724325793,not_relevant,1.0000,Damn the Stingray ! Chevy gots to have the bes...
4664,724325819,not_relevant,0.7708,Can you guys picture my little 5 foot tall sel...
4684,724325839,not_relevant,0.2480,@samuelfine I very rarely drive for any distan...
4844,724325999,not_relevant,1.0000,I should add that my new used car is self-driv...
5061,724326216,not_relevant,0.7653,I threw a self-driving dart through a cloud th...


In [20]:
# Re-making tuples
subset = train_set[['text', 'sentiment']]
tuples = [tuple(x) for x in subset.values]

len(tuples)

5009

In [21]:
# Tokenizing words
all_words = toktok.tokenize(subset[['text']] )

s = [({word: (word in (x[0])) for word in all_words}, x[1]) for x in tuples]
len(s)

5009

In [44]:
# Naive Bayes
classifier = nltk.NaiveBayesClassifier.train(s)
classifier.show_most_informative_features(20)

Most Informative Features
                   threw = True           not_re : releva =     54.0 : 1.0
                  Chrome = True           not_re : releva =     32.4 : 1.0
                tracking = True           not_re : releva =     32.4 : 1.0
         _______________ = True           not_re : releva =     32.4 : 1.0
                     Dor = True           not_re : releva =     32.4 : 1.0
               surprised = True           not_re : releva =     32.4 : 1.0
                  Second = True           not_re : releva =     32.4 : 1.0
                   waved = True           not_re : releva =     32.4 : 1.0
                     SAE = True           not_re : releva =     32.4 : 1.0
                 On-Road = True           not_re : releva =     32.4 : 1.0
                   Chevy = True           not_re : releva =     32.4 : 1.0
                   view. = True           not_re : releva =     32.4 : 1.0
               Standards = True           not_re : releva =     19.4 : 1.0

In [23]:
# Linear SVC
LinearSVC_classifier = SklearnClassifier(LinearSVC())
LinearSVC_classifier.train(s)

<SklearnClassifier(LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0))>

In [24]:
print "Percent Accuracy:"
print "------------------"
print "Naive Bayes: %d" %round((nltk.classify.accuracy(classifier, s))*100, 2)
print "LinearSVC: %d" %round((nltk.classify.accuracy(LinearSVC_classifier, s))*100, 2)

Percent Accuracy:
------------------
Naive Bayes: 91
LinearSVC: 97


#### Using the models on the final testing set

In [25]:
subset2 = final_set[['text', 'sentiment']]
tuples2 = [tuple(x) for x in subset2.values]
all_words2 = toktok.tokenize(subset2[['text']])
s2 = [({word: (word in (x[0])) for word in all_words2}, x[1]) for x in tuples2]

print "Percent Accuracy:"
print "------------------"
print "Naive Bayes: %d" %round((nltk.classify.accuracy(classifier, s2))*100, 2)
print "LinearSVC: %d" %round((nltk.classify.accuracy(LinearSVC_classifier, s2))*100, 2)

Percent Accuracy:
------------------
Naive Bayes: 96
LinearSVC: 87


**Conclusion:** The Linear SVC accuracy dropped for the testing dataset, while the Naive Bayes model inccreased. The potential reason for this may be due to how the models work. Naive Bayes relies on classifying text based on each word's probablity while the Linear SVC model determines classification using a "best fit" approach to a collection of words. When determining if a topic is not relevant, it may be difficult to classify relevancy due to the abundance and sheer randomness of non-relevant terms. Overall, classifying non-relevant terms as "not_relevant" may be harder than determining the classification between two distinct groups simply because something "not_relevant" can be completely different from the topic at hand or close enough to the topic to mention it as a tangent.